In [2]:
%pip install yfinance pandas numpy matplotlib seaborn nltk textblob tensorflow scikit-learn newsapi-python yahoo_fin GoogleNews GoogleNews-v2


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement GoogleNews-v2 (from versions: none)
ERROR: No matching distribution found for GoogleNews-v2


In [20]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from GoogleNews import GoogleNews
import datetime

In [33]:
stock_data = yf.download('AAPL', start='2022-01-01', end='2024-01-01')[['Close']].copy()
stock_data = stock_data.reset_index()  # 'Date' becomes a column
stock_data.columns = ['date', 'Close']  # Rename explicitly
stock_data['date'] = pd.to_datetime(stock_data['date']).dt.date  # Ensure format

[*********************100%***********************]  1 of 1 completed


In [34]:
# 2. Fetch news and calculate sentiment
googlenews = GoogleNews(lang='en', period='30d')  # last 30 days
googlenews.search('Apple stock')

# for i in range(2, 6):
#     googlenews.get_page(i)

sentiments = []
dates = []
results = googlenews.results()

for article in results:
    text = article['title'] + '. ' + article.get('desc', '')
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    date_str = article.get('date', 'now')

    try:
        date_obj = datetime.datetime.strptime(date_str, '%b %d, %Y')
    except:
        date_obj = datetime.datetime.today()

    sentiments.append(polarity)
    dates.append(date_obj.date())

sentiment_df = pd.DataFrame({'date': dates, 'sentiment': sentiments})
sentiment_df = sentiment_df.groupby('date').mean().reset_index()
sentiment_df['date'] = pd.to_datetime(sentiment_df['date']).dt.date


In [35]:
merged = pd.merge(stock_data, sentiment_df, on='date', how='left')
merged['sentiment'] = merged['sentiment'].fillna(method='ffill').fillna(0)
merged.set_index('date', inplace=True)

C:\Users\ayu\AppData\Local\Temp\ipykernel_17560\3595295240.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged['sentiment'] = merged['sentiment'].fillna(method='ffill').fillna(0)


In [40]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(merged[['Close', 'sentiment']])

# 5. Create LSTM sequences
def create_sequences(data, seq_len=60):
    x = []
    y = []
    for i in range(seq_len, len(data)):
        x.append(data[i-seq_len:i])
        y.append(data[i, 0])  # Predicting 'Close'
    return np.array(x), np.array(y)

x, y = create_sequences(scaled)

In [41]:
split = int(0.8 * len(x))
x_train, y_train = x[:split], y[:split]
x_test, y_test = x[split:], y[split:]


In [42]:
# 7. LSTM model using price + sentiment
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=20, batch_size=32)


Epoch 1/20


c:\Users\ayu\StockPrediction\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.1874
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0279
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0164
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0117
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0115
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0094
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0089
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0081
Epoch 9/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0076
Epoch 10/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0092
Epoch 11/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0077
Epoch 12/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0084
Epoch 13/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0073
Epoch 14/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0080
Epoch 15/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0077
Epoch 16/20
11/

In [43]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
real = scaler.inverse_transform(y_test.reshape(-1, 1))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step


ValueError: non-broadcastable output operand with shape (89,1) doesn't match the broadcast shape (89,2)

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(real, color='blue', label='Actual Price')
plt.plot(predictions, color='red', label='Predicted Price')
plt.legend()
plt.title('Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.show()


In [ ]:
combined_data = np.concatenate((scaled_data, sentiment_array), axis=1)
